In [ ]:
import os
import torch
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
from torchvision import transforms
from scipy.spatial import distance  # To calculate Euclidean distance

# Load the pre-trained model
model = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet50', pretrained=True)
model.eval()

# Preprocessing pipeline
preprocess = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Function to extract points on vertical and horizontal lines
def extract_line_points(mask, num_vertical=10, num_horizontal=5):
    height, width = mask.shape
    
    # Extract points along vertical lines
    vertical_lines_points = []
    for i in range(1, num_vertical + 1):
        x = width * i // (num_vertical + 1)  # Calculate x-coordinate
        line_points = [(x, y) for y in range(height) if mask[y, x] > 0]  # Get non-zero points
        vertical_lines_points.append(line_points)
    
    # Extract points along horizontal lines
    horizontal_lines_points = []
    for i in range(1, num_horizontal + 1):
        y = height * i // (num_horizontal + 1)  # Calculate y-coordinate
        line_points = [(x, y) for x in range(width) if mask[y, x] > 0]  # Get non-zero points
        horizontal_lines_points.append(line_points)
    
    return vertical_lines_points, horizontal_lines_points

# Function to find the maximum Euclidean distance between points on lines
def find_max_euclidean_distance(points):
    max_distance = 0
    point_pair = None

    # Iterate over all pairs of points and calculate the Euclidean distance
    for i in range(len(points)):
        for j in range(i + 1, len(points)):
            p1, p2 = points[i], points[j]
            dist = distance.euclidean(p1, p2)
            if dist > max_distance:
                max_distance = dist
                point_pair = (p1, p2)
    
    return point_pair, max_distance

# Function to process a single image
def process_image(image_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    input_image = Image.open(image_path).convert("RGB")
    original_image = np.array(input_image)
    input_tensor = preprocess(input_image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)['out'][0]
    output_predictions = output.argmax(0).cpu().numpy()

    # Refine segmentation mask
    segmentation_mask = (output_predictions > 0).astype(np.uint8)

    # Step 1: Extract points on vertical and horizontal lines
    vertical_lines_points, horizontal_lines_points = extract_line_points(segmentation_mask)

    # Step 2: Find the maximum Euclidean distance points (vertically and horizontally)
    max_vertical_pair = None
    max_horizontal_pair = None
    max_vertical_distance = 0
    max_horizontal_distance = 0

    for vertical_points in vertical_lines_points:
        if len(vertical_points) > 1:
            vertical_pair, vertical_distance = find_max_euclidean_distance(vertical_points)
            if vertical_distance > max_vertical_distance:
                max_vertical_distance = vertical_distance
                max_vertical_pair = vertical_pair
    
    for horizontal_points in horizontal_lines_points:
        if len(horizontal_points) > 1:
            horizontal_pair, horizontal_distance = find_max_euclidean_distance(horizontal_points)
            if horizontal_distance > max_horizontal_distance:
                max_horizontal_distance = horizontal_distance
                max_horizontal_pair = horizontal_pair

    # Step 3: Mark the key points on the image
    mask_colored = apply_color_map(output_predictions)

    if max_vertical_pair:
        cv2.circle(mask_colored, max_vertical_pair[0], radius=5, color=(0, 255, 0), thickness=-1)  # First point in green
        cv2.circle(mask_colored, max_vertical_pair[1], radius=5, color=(0, 255, 0), thickness=-1)  # Second point in green

    if max_horizontal_pair:
        cv2.circle(mask_colored, max_horizontal_pair[0], radius=5, color=(0, 255, 0), thickness=-1)  # First point in green
        cv2.circle(mask_colored, max_horizontal_pair[1], radius=5, color=(0, 255, 0), thickness=-1)  # Second point in green

    # Step 4: Show the results
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(original_image)
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.title('Segmented Mask with Key Points')
    plt.imshow(mask_colored)
    plt.axis('off')

    plt.show()

    # Output the distances and points
    if max_vertical_pair:
        print(f"Max Vertical Euclidean Distance: {max_vertical_distance}")
        print(f"Points: {max_vertical_pair}")

    if max_horizontal_pair:
        print(f"Max Horizontal Euclidean Distance: {max_horizontal_distance}")
        print(f"Points: {max_horizontal_pair}")


def process_images_in_folder(folder_path, limit=3):
    count = 0  # Initialize a counter for the number of images processed
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)
        if image_name.endswith(('.png', '.jpg', '.jpeg')):  
            print(f"Processing: {image_name}")
            process_image(image_path)
            count += 1  # Increment the counter
            if count >= limit:  # Stop after processing 3 images
                break


folder_path = r'D:\Office-Work\New folder\bbox-yolov5-cattle-top-view.v1i.coco\train'  
process_images_in_folder(folder_path)
